# Spectroscopic selection degrader to emulate zCOSMOS training samples

last run successfully: April 26, 2023

The spectroscopic_selection degrader can be used to model the spectroscopic success rates in training sets based on real data.  Given a 2-dimensional grid of spec-z success ratio as a function of two variables (often magnitude, color, or redshift), the degrader will draw the appropriate fraction of samples from the input data and return a sample with incompleteness modeled.  

The degrader takes the following arguments:
- `N_tot`: number of selected sources <br>
- `nondetect_val`: non detected magnitude value to be excluded (usually 99.0, -99.0 or NaN).<br>
- `downsample`: If true, downsample the selected sources into a total number of N_tot. <br>
- `success_rate_dir`: The path to the directory containing success rate files.
- `colnames`: a dictionary that includes necessary columns (magnitudes, colors and redshift) for selection. For magnitudes, the keys are ugrizy; for colors, the keys are, for example, gr standing for g-r; for redshift, the key is 'redshift'. In this demo, zCOSMOS takes {'i':'i', 'redshift':'redshift'} as minimum necessary input

In this quick notebook we'll select galaxies based on zCOSMOS selection function.

In [ ]:
import rail
import os
import matplotlib.pyplot as plt
import numpy as np
import tables_io
import pandas as pd
#from rail.core.data import TableHandle
from rail.core.stage import RailStage
%matplotlib inline 

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

Let's make fake data for zCOSMOS selection. 

In [ ]:
i = np.random.uniform(low=18, high=25.9675, size=(2000000,))
gz = np.random.uniform(low=-1.98, high=5.98, size=(2000000,))
u = np.full_like(i, 20.0, dtype=np.double)
g = np.full_like(i, 20.0, dtype=np.double)
r = np.full_like(i, 20.0, dtype=np.double)
y = np.full_like(i, 20.0, dtype=np.double)
z = g - gz
redshift = np.random.uniform(size=len(i)) * 2

standardize the column names

In [ ]:
mockdict = {}
for label, item in zip(['u', 'g','r','i', 'z','y', 'redshift'], [u,g,r,i,z,y, redshift]):
    mockdict[f'{label}'] = item

np.repeat(item, 100).flatten()

In [ ]:
df = pd.DataFrame(mockdict)

In [ ]:
df.head()

Now, let's import the spectroscopic_selections degrader for zCOSMOS. <br>
The ratio file for zCOSMOS is located in the `RAIL/src/rail/examples/creation/data/success_rate_data/` directory, as we are in `RAIL/examples/creation` folder named `zCOSMOS_success.txt`; the binning in i band and redshift are given in `zCOSMOS_I_sampling.txt` and `zCOSMOS_z_sampling.txt`.<br>
We will set a random seed for reproducibility, and set the output file to write our incomplete catalog to "test_hsc.pq".

In [ ]:
import sys
from rail.creation.degradation import spectroscopic_selections
from importlib import reload
from rail.creation.degradation.spectroscopic_selections import SpecSelection_zCOSMOS

In [ ]:
zcosmos_selecter = SpecSelection_zCOSMOS.make_stage(downsample=False, 
                                                    colnames={'i':'i','redshift':'redshift'})

Let's run the code and see how long it takes:

In [ ]:
%%time
trim_data = zcosmos_selecter(df)

In [ ]:
trim_data.data.info()

And we see that we've kept 503967 out of the 2,000,000 galaxies in the initial sample, so about 25% of the initial sample.  To visualize our cuts, let's read in the success ratios file and plot our sample overlaid with an alpha of 0.05, that way the strength of the black dot will give a visual indication of how many galaxies in each cell we've kept.

In [ ]:
# compare to sum of ratios * 100
ratio_file='../../src/rail/examples_data/creation_data/data/success_rate_data/zCOSMOS_success.txt'

In [ ]:
ratios = np.loadtxt(ratio_file)

In [ ]:
ibin_ = np.arange(18, 22.4, 0.01464226, dtype=np.float64)
zbin_ = np.arange(0, 1.4, 0.00587002, dtype=np.float64)

ibin, zbin = np.meshgrid(ibin_, zbin_)

In [ ]:
trim_data.data

In [ ]:
plt.figure(figsize=(12,12))
plt.title('zCOSMOS', fontsize=20)

c = plt.pcolormesh(zbin, ibin, ratios.T, cmap='turbo',vmin=0, vmax=1, alpha=0.8)
plt.scatter(trim_data.data['redshift'], trim_data.data['i'], s=2, c='k',alpha =.05)
plt.xlabel("redshift", fontsize=15)
plt.ylabel("i band Magnitude", fontsize=18)
cb = plt.colorbar(c, label='success rate',orientation='horizontal', pad=0.1)
cb.set_label(label='success rate', size=15)

The colormap shows the zCOSMOS success ratios and the strenth of the black dots shows how many galaxies were actually kept. We see perfect agreement between our predicted ratios and the actual number of galaxies kept, the degrader is functioning properly, and we see a nice visual representation of the resulting spectroscopic sample incompleteness.